# Model Selection Using the HBM

This script automates the process of permuting through all possible models (sets of mappings between EO/ER and HBM's four constructs) and filtering based on the desired p(behaviour) ordering. The script prints out the mappings in models which adhere to the p(behaviour) ordering. 

In [2]:
import itertools
from collections import defaultdict, OrderedDict
import matplotlib.pyplot as plt

# Define values used for OR 
OR_sus=1.11
OR_sev=1.62
OR_ben=1.72
OR_bar=0.86
OR0=0.35

In [3]:
# Function which calculates p(behaviour) based on the logistic regression equation
def calculate_probability(x0, x1, x2, x3):
    p = (OR0 * OR_sus**x0 * OR_sev**x1 * OR_ben**x2 * OR_bar**x3) / (1 + (OR0 * OR_sus**x0 * OR_sev**x1 * OR_ben**x2 * OR_bar**x3))
    return p

In [5]:
# Find all permutations of EO and ER effects on the four constructs of the HBM 
possible_variable_effects = set(itertools.product(["EO", "ER"], repeat=4)) - set([('ER', 'ER', 'ER', 'ER'), ('EO', 'EO', 'EO', 'EO')])

# Dictionary used for printing mappings later 
dict = {
    0: "susceptibility",
    1: "severity",
    2: "benefits",
    3: "barriers"
}

# Keep count of total number of combinations (count) and number of valid combinations (perm_count)
count = 0
perm_count = 0

# Iterate through all permutations and print out valid combinations 
for comb in possible_variable_effects:
    
    # Produce all permutations available with the number of times EO and ER each affects constructs
    num_education = comb.count("EO")
    num_income = comb.count("ER")
    education_mappings = itertools.product(itertools.permutations([0, 1]), repeat=num_education)
    income_mappings = itertools.product(itertools.permutations([0, 1]), repeat=num_income)
    
    # The final set of mappings for this set of variable effects 
    combined_mappings = itertools.product(education_mappings, income_mappings)
    mapping_set = set()
    
    # For each mapping, validate it against the probability inequality 
    for mapping in combined_mappings:
        i_edu = 0
        i_inc = 0
        x_low_education_low_income = []
        x_high_education_low_income = []
        x_low_education_high_income = []
        x_high_education_high_income = []
        final_mapping = []
        
        # Find x values for each construct for each sub-population group 
        for i in range(len(comb)):
            if comb[i] == "EO":
                x = mapping[0][i_edu]
                x_low_education_low_income.append(x[0])
                x_high_education_low_income.append(x[1])
                x_low_education_high_income.append(x[0])
                x_high_education_high_income.append(x[1])
                i_edu += 1
            else:
                x = mapping[1][i_inc]
                x_low_education_low_income.append(x[0])
                x_high_education_low_income.append(x[0])
                x_low_education_high_income.append(x[1])
                x_high_education_high_income.append(x[1])
            final_mapping.append(x)
                
        # Calculate p(behaviour) value for each sub-population group 
        low_education_low_income = calculate_probability(x_low_education_low_income[0], x_low_education_low_income[1], x_low_education_low_income[2], x_low_education_low_income[3])
        high_education_low_income = calculate_probability(x_high_education_low_income[0], x_high_education_low_income[1], x_high_education_low_income[2], x_high_education_low_income[3])
        high_education_high_income = calculate_probability(x_high_education_high_income[0], x_high_education_high_income[1], x_high_education_high_income[2], x_high_education_high_income[3])
        low_education_high_income = calculate_probability(x_low_education_high_income[0], x_low_education_high_income[1], x_low_education_high_income[2], x_low_education_high_income[3])        
        perm_count += 1
        
        # Only print the result if the probability inequality follows the order in observed mobility trend
        if high_education_low_income >= high_education_high_income >= low_education_low_income >= low_education_high_income:
            final_mapping = tuple(final_mapping)
            if final_mapping not in mapping_set:
                count += 1
                mapping_set.add(final_mapping)
                print(comb, final_mapping)
                print(high_education_low_income, high_education_high_income, low_education_low_income, low_education_high_income)
                for i in range(len(comb)):
                    level = "high" if final_mapping[i][1] == 1 else "low"
                    mapping_str = "High " + comb[i] + ": " + level + " " + dict[i]
                    print(mapping_str)
                print("\n")
                
print("The number of models with desired probability inequality: ", count)
print("The total number of models: ", perm_count)

('ER', 'EO', 'EO', 'EO') ((1, 0), (0, 1), (0, 1), (0, 1))
0.48212398255954325 0.4561393814694939 0.27979834353619015 0.25925925925925924
High ER: low susceptibility
High EO: high severity
High EO: high benefits
High EO: high barriers


('ER', 'EO', 'EO', 'EO') ((1, 0), (0, 1), (0, 1), (1, 0))
0.5198117047241501 0.4937324072011502 0.2504366206684607 0.23136049192928518
High ER: low susceptibility
High EO: high severity
High EO: high benefits
High EO: low barriers


('ER', 'EO', 'EO', 'EO') ((1, 0), (1, 0), (0, 1), (1, 0))
0.4005586793108823 0.3757802746566792 0.3511794454686437 0.32778532152028067
High ER: low susceptibility
High EO: low severity
High EO: high benefits
High EO: low barriers


('EO', 'EO', 'EO', 'ER') ((0, 1), (0, 1), (0, 1), (0, 1))
0.5198117047241501 0.48212398255954325 0.25925925925925924 0.23136049192928518
High EO: high susceptibility
High EO: high severity
High EO: high benefits
High ER: high barriers


('EO', 'EO', 'EO', 'ER') ((0, 1), (1, 0), (0, 1), (0, 1))
0.40